In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from glob import glob
from tqdm import tqdm
tqdm.pandas()

import cv2

import tensorflow_hub as hub
import numpy as np

import efficientnet.tfkeras as efn 
import random

In [2]:
def get_frames(path, frames_denominator=2):
    """
    Get frames for the video at the given path
    frames_denominator specifies if frames should be dropped, pass 1 for no dropping, 2 to drop every other, 3 to drop 2/3, etc.
    """
    vidcap = cv2.VideoCapture(path)
    success,image = vidcap.read()
    frames = []
    count = 0
    while success:
        if (count % frames_denominator) == 0:
            frames.append(image)   
        success,image = vidcap.read()
        count += 1
    return frames

def center_crop(image, center_shape=(448, 448)):
    side_crops = (image.shape[0] - center_shape[0]) // 2
    top_n_bottom_crops = (image.shape[1] - center_shape[1]) // 2
    cropped_image = image[side_crops:-side_crops,top_n_bottom_crops:-top_n_bottom_crops]
    return cropped_image

from skimage.transform import resize
def process_frame(frame, precrop_shape=(448, 448), input_shape=(224, 224)):
    """
    Crop a frame down into the precrop shape, then downscape to the given input shape
    """
    cropped = center_crop(frame, center_shape=precrop_shape) /255
    resized_image = resize(
        cropped,
        input_shape,
        order=3, # Bicubic interpolation
        preserve_range=True,
    )
    return resized_image

# Model Plan
- CNN + LSTM model
- CNN will be a pretrained efficientnet (https://arxiv.org/pdf/1905.11946.pdf)
- LSTM will be trained from scratch for ease of implementation

In [3]:
files = glob("../data/clips/*/*.mp4")

In [4]:
clip_df = pd.DataFrame()
clip_df["path"] = files
clip_df["label"] = clip_df["path"].apply(lambda x : x.split("/")[-2])
clip_df.head()

,path,label
0,../data/clips/normal/41.mp4,normal
1,../data/clips/normal/43.mp4,normal
2,../data/clips/normal/12.mp4,normal
3,../data/clips/normal/38.mp4,normal
4,../data/clips/normal/14.mp4,normal


In [5]:
clip_df["label"].value_counts()

hax       35
normal    35
Name: label, dtype: int64

In [6]:
# CNN_TRAINABLE = False
# Batch_size, time_component, length, width, depth
input_shape = (60, 224, 224, 3)

In [7]:
efn_b0 = efn.EfficientNetB0(weights='imagenet')

In [8]:
inputs = tf.keras.Input(shape=input_shape)
embedding_sequence = tf.keras.layers.TimeDistributed(efn_b0)(inputs)
gru_layer = tf.keras.layers.GRU(10)(embedding_sequence)
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(gru_layer)

In [9]:
model = tf.keras.Model(inputs, output_layer)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 60, 224, 224, 3)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 60, 1000)          5330564   
_________________________________________________________________
gru (GRU)                    (None, 10)                30360     
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 5,360,935
Trainable params: 5,318,919
Non-trainable params: 42,016
_________________________________________________________________


In [10]:
def get_items_loop(lst, low, high):
    nums = range(low, high)
    nums = [num % len(lst) for num in nums]
    return [lst[idx] for idx in nums]


# TODO fix batch_size not actually being batch_size
def data_generator(pos_paths, neg_paths, batch_size=8):
    assert batch_size / 2 != 0, "Batch size must be a multiple of 2 for balanced data generation"
    total_size = len(pos_paths + neg_paths)
    batch_size = batch_size//2
    while True:
        random.shuffle(pos_paths)
        random.shuffle(neg_paths)
        count = 0
        while count < total_size:
            combined_paths = get_items_loop(pos_paths, count, count+batch_size)
            combined_paths += get_items_loop(neg_paths, count, count+batch_size)
            frame_list = [[process_frame(frame) for frame in get_frames(path)] for path in combined_paths]
            count += batch_size*2
            labels = np.zeros(batch_size*2)
            labels[0:batch_size] = 1
            yield np.stack(frame_list), np.array(labels)

In [11]:
hax_paths = clip_df[clip_df["label"] == "hax"]["path"].tolist()
normal_paths = clip_df[clip_df["label"] == "normal"]["path"].tolist()
len(hax_paths)

35

In [12]:
batch_size = 2

In [13]:
dg = data_generator(hax_paths, normal_paths, batch_size=batch_size)

In [14]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
model.fit_generator(dg, epochs=10, steps_per_epoch=86//batch_size)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
43/43 [==============================] - 108s 2s/step - loss: 0.6868 - accuracy: 0.6163
Epoch 2/10
43/43 [==============================] - 98s 2s/step - loss: 0.6552 - accuracy: 0.7674
Epoch 3/10
43/43 [==============================] - 99s 2s/step - loss: 0.5673 - accuracy: 0.7674
Epoch 4/10
43/43 [==============================] - 101s 2s/step - loss: 0.3287 - accuracy: 0.9070
Epoch 5/10
43/43 [==============================] - 99s 2s/step - loss: 0.1022 - accuracy: 1.0000
Epoch 6/10
43/43 [==============================] - 99s 2s/step - loss: 0.0637 - accuracy: 1.0000
Epoch 7/10
43/43 [==============================] - 101s 2s/step - loss: 0.2660 - accuracy: 0.9070
Epoch 8/10
43/43 [==============================] - 103s 2s/step - loss: 0.0835 - accuracy: 0.9884
Epoch 9/10
43/43 [==============================] - 102s 2s/step - loss: 0.1022 - accuracy: 0.9767
Epoch 10/10
43/43 [==============================] - 101s 2s/step - loss: 0.4979 - accuracy: 0.8140


In [16]:
len(hax_paths)

35

In [17]:
ho_files = glob("../data/holdout_clips/*/*.mp4")
holdout_df = pd.DataFrame()
holdout_df["path"] = ho_files
holdout_df["label"] = holdout_df["path"].apply(lambda x : x.split("/")[-2])
holdout_df.head()

,path,label
0,../data/holdout_clips/normal/20.mp4,normal
1,../data/holdout_clips/normal/35.mp4,normal
2,../data/holdout_clips/normal/10.mp4,normal
3,../data/holdout_clips/normal/5.mp4,normal
4,../data/holdout_clips/normal/40.mp4,normal


In [18]:
holdout_df["label"].value_counts()

hax       8
normal    8
Name: label, dtype: int64

In [19]:
ho_hax_paths = holdout_df[holdout_df["label"] == "hax"]["path"].tolist()
ho_normal_paths = holdout_df[holdout_df["label"] == "normal"]["path"].tolist()

In [20]:
ho_normal_paths

['../data/holdout_clips/normal/20.mp4',
 '../data/holdout_clips/normal/35.mp4',
 '../data/holdout_clips/normal/10.mp4',
 '../data/holdout_clips/normal/5.mp4',
 '../data/holdout_clips/normal/40.mp4',
 '../data/holdout_clips/normal/15.mp4',
 '../data/holdout_clips/normal/30.mp4',
 '../data/holdout_clips/normal/25.mp4']

In [21]:
steps = (len(ho_hax_paths) + len(ho_normal_paths)) / batch_size

In [22]:
holdout_generator = data_generator(ho_hax_paths, ho_normal_paths, batch_size=batch_size)

In [23]:
model.evaluate(holdout_generator, steps=steps)

8/8 [==============================] - 18s 2s/step - loss: 0.6794 - accuracy: 0.7500


[0.6793551445007324, 0.75]

In [24]:
inp, out = next(holdout_generator)

In [25]:
out

array([1., 0.])

In [26]:
model.predict(inp)

array([[0.9674691 ],
       [0.06465961]], dtype=float32)